In [75]:
import pandas as pd
import numpy as np
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

pit_cols = ['시즌', '이름', '생년월일', '팀', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드', '이닝', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
bat_cols = ['시즌', '이름', '생년월일', '팀', '포지션', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타','타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR', 'WPA']
position_list = ['1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF', 'DH', 'C']



DRIVER_PATH = 'C:\\Users\\USER\\chromedriver-win64\\chromedriver.exe'

In [119]:
def team_named(df):
    team_dict = dict(zip(['두', '키', 'L', 'S', '삼', '한', '롯', 'N', 'KIA', 'KT', '넥', 'k'],['두산', '키움', 'LG', 'SSG', '삼성', '한화', '롯데', 'NC', 'KIA', 'KT', '넥센', 'KT']))

    df['팀'] = df['팀'].apply(lambda teams : [team_dict[team] for team in teams] if isinstance(teams, list) else team_dict[teams])
    
    return df

def crawling_statiz_batter(ys, ye, by_sort,length):
    site = f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys={ys}&ye={ye}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1={by_sort}&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn={length}&si=&cn='
    response = requests.get(site)
    soup = BeautifulSoup(response.text, 'html.parser')

    tr_tags = soup.select('table#mytable > tr')
    i=0
    player_infos=[]

    for tr_tag in tr_tags:
        if (i%12==0) or (i%12==1):
            i+=1
            continue
        birth = tr_tag.select_one('a')['href'].split('birth=')[1] #생년월일
        name = tr_tag.select_one('a').get_text() #이름

        td_tags = tr_tag.select('td')
        year = list(td_tags[2].strings)[0] # 시즌
        position = [element for element in list(td_tags[2].strings) if element in position_list]
        position.append(np.nan)
        position = position[0] #포지션

        # 팀
        # 팀이 여러개일 경우를 대비
        # KT, KIA는 같은 K이므로 스타일을 봐야함
        if len([element for element in list(td_tags[2].strings) if element not in position_list]) == 2:
            team = [element for element in list(td_tags[2].strings) if element not in position_list][1]
            if team == 'K':
                if td_tags[2].select('span > span')[1]['style'].split(';')[2] == 'background:black':
                    team = 'KT'
                else :
                    team = 'KIA'
        else:
            team = list(td_tags[2].strings)
            k_index = [i for i, value in enumerate(list(td_tags[2].strings)) if value == 'K']
            for index in k_index:
                if td_tags[2].select('span > span')[index]['style'].split(';')[2] == 'background:black':
                    team[index] = 'KT'
                else:
                    team[index] = 'KIA'
            team = team[1:-1]
            

        player_info=[year, name, birth, team, position]

        for td_tag in td_tags[4:]:
            player_info.append(td_tag.get_text())

        player_infos.append(player_info)
        i+=1
    
    df = pd.DataFrame(player_infos,columns=bat_cols)
    
    df[['시즌', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비']] = df[['시즌', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비']].astype(int)
    
    df[['타율', '출루', '장타', 'OPS','wOBA']] = df[['타율', '출루', '장타', 'OPS','wOBA']].applymap(lambda x : float(x.replace('.', ''))/1000)
    df[['wRC+', 'WAR', 'WPA']] = df[['wRC+', 'WAR', 'WPA']].applymap(lambda x: float(x))
    df['생년월일'] = pd.to_datetime(df['생년월일'])
    
    df = team_named(df)
    
    df['나이']=df['시즌']+2000-df['생년월일'].dt.year+1
    
    return df


def crawling_statiz_pitcher(ys, ye, by_sort,length):
    site = f'http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys={ys}&ye={ye}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1={by_sort}&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn={length}&si=&cn='
    response = requests.get(site)
    soup = BeautifulSoup(response.text, 'html.parser')

    tr_tags = soup.select('table#mytable > tr')
    tr_tags
    i=0
    player_infos=[]
    for tr_tag in tr_tags:
        if (i%12==0) or (i%12==1):
            i+=1
            continue
        birth = tr_tag.select_one('a')['href'].split('birth=')[1] #생년월일
        name = tr_tag.select_one('a').get_text() #이름
        td_tags = tr_tag.select('td')
        year = list(td_tags[2].strings)[0] # 시즌

        #팀
        #팀이 여러개일 경우 대비
        #KT, KIA 구분
        if len([element for element in list(td_tags[2].strings)]) == 2:
            team = [element for element in list(td_tags[2].strings)][1]
            if team == 'K':
                if td_tags[2].select('span > span')[1]['style'].split(';')[2] == 'background:black':
                    team = 'KT'
                else:
                    team = 'KIA'
        else:
            team = list(td_tags[2].strings)
            k_index = [i for i, value in enumerate(list(td_tags[2].strings)) if value == 'K']
            for index in k_index:
                if td_tags[2].select('span > span')[index]['style'].split(';')[2] == 'background:black':
                    team[index] = 'KT'
                else:
                    team[index] = 'KIA'
            team = team[1:]

        player_info=[year, name, birth, team]

        for td_tag in td_tags[4:]:
            player_info.append(td_tag.get_text())

        player_infos.append(player_info)
        i+=1

        df = pd.DataFrame(player_infos,columns=pit_cols)
        df.drop(columns=['ERA+', 'FIP+'], inplace=True)

        df[['시즌', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투']] = df[['시즌', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투']].astype(int)
#         df[['이닝', 'ERA', 'FIP', 'WHIP', 'WAR', 'WPA']] = df[['이닝', 'ERA', 'FIP', 'WHIP', 'WAR', 'WPA']].applymap(lambda x : float(x.replace('.', ''))/1000)
        df[['이닝', 'ERA', 'FIP', 'WHIP', 'WAR', 'WPA']] = df[['이닝', 'ERA', 'FIP', 'WHIP', 'WAR', 'WPA']].astype('float')
        df['생년월일'] = pd.to_datetime(df['생년월일'])

        df = team_named(df)
        df['나이']=df['시즌']+2000-df['생년월일'].dt.year+1
        
    return df

In [125]:
bat_df=pd.DataFrame()
for i in range(2014,2024):
    year_df = crawling_statiz_batter(i,i,'GP',200)
    bat_df = pd.concat([bat_df,year_df],axis=0).reset_index(drop=True)

In [126]:
pit_df=pd.DataFrame()
for i in range(2014,2024):
    year_df = crawling_statiz_pitcher(i, i, 'OutCount', '200')
    pit_df = pd.concat([pit_df,year_df],axis=0).reset_index(drop=True)

In [130]:
# bat_df.to_csv('프로야구 데이터/14시즌~23시즌 타자 데이터.csv')
# pit_df.to_csv('프로야구 데이터/14시즌~23시즌 투수 데이터.csv')